# Drought Prediction

## Load Libraries

In [2]:
# Drought Prediction
## Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, metrics
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay, classification_report,
                             accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, auc, cohen_kappa_score)
from sklearn.naive_bayes import GaussianNB
import pickle
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NeighbourhoodCleaningRule, NearMiss
# Import PyTorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchviz import make_dot
import random

## Data Wrangling

In [3]:
drought_df =  pd.read_csv('data/all_timeseries.csv')

with open('data/Xy_trainTest.pkl', 'rb') as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

In [4]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [5]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Define Neural Network Models

In [25]:
# Base Model
class DroughtNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Initialize the DroughtNet model.

        Parameters:
            input_dim (int): The number of input features.
            hidden_dim (int): The number of neurons in the hidden layers.
            output_dim (int): The number of output classes.
        """
        super(DroughtNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)     # First fully connected layer
        self.relu = nn.ReLU()                           # ReLU activation function
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)    # Second fully connected layer
        self.fc3 = nn.Linear(hidden_dim, output_dim)    # Output layer
        
    def forward(self, x):
        """
        Define the forward pass of the model.

        Parameters:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output of the model.
        """
        out = self.fc1(x)       # Apply first fully connected layer
        out = self.relu(out)    # Apply ReLU activation
        out = self.fc2(out)     # Apply second fully connected layer
        out = self.relu(out)    # Apply ReLU activation
        out = self.fc3(out)     # Apply output layer
        return out

# Complex Model
class DroughtNetComplex(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        """
        Initialize the DroughtNetComplex model.

        Parameters:
            input_dim (int): The number of input features.
            hidden_dims (list of int): A list where each element specifies the number of neurons in that hidden layer.
            output_dim (int): The number of output classes.
        """
        super(DroughtNetComplex, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dims[0])                                                                 # Input layer
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dims[i], hidden_dims[i+1]) for i in range(len(hidden_dims)-1)])    # Hidden layers
        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)                                                              # Output layer
        self.relu = nn.ReLU()                                                                                                   # ReLU activation function

    def forward(self, x):
        """
        Define the forward pass of the model. 

        Parameters:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output of the model.
        """
        out = self.relu(self.input_layer(x))    # Apply input layer and ReLU activation
        for layer in self.hidden_layers:
            out = self.relu(layer(out))         # Apply each hidden layer and ReLU activation
        out = self.output_layer(out)            # Apply output layer
        return out

# Train Model
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, patience=3):
    """
    Train the given model. Early stopping based on validation loss.

    Parameters:
        model (nn.Module): The neural network model to train.
        train_loader (DataLoader): DataLoader for the training data.
        test_loader (DataLoader): DataLoader for the test/validation data.
        criterion (nn.Module): Loss function.
        optimizer (optim.Optimizer): Optimizer for the model.
        num_epochs (int): Number of epochs to train the model.
        patience (int): Number of epochs to wait if validation loss stops decreasing.

    Returns:
        nn.Module: Best model based on validation loss.
        dict: A dictionary containing the training history (loss and accuracy for training and validation).
    """

    # Initialize variables for early stopping
    best_val_loss = float('inf')# Set initial best validation loss to infinity
    current_patience = 0        # Initialize patience counter
    best_model = None           # Initialize variable to store the best model
    best_train_history = None   # Initialize variable to store the training history of the best model

    train_history = {'loss': [], 'accuracy': [], 'validation_loss': [], 'validation_accuracy': []}

    for epoch in range(num_epochs):
        # Set model to training mode, Initialize variables
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()                       # Zero the parameter gradients
            outputs = model(X_batch)                    # Forward pass
            loss = criterion(outputs, y_batch)          # Compute loss
            loss.backward()                             # Backward pass
            optimizer.step()                            # Optimize the weights
            running_loss += loss.item() * X_batch.size(0)  # Accumulate running loss
            
            _, predicted = torch.max(outputs, 1)            # Get predictions
            correct += (predicted == y_batch).sum().item()  # Count correct predictions
            total += y_batch.size(0)                        # Count total samples
        
        epoch_loss = running_loss / len(train_loader.dataset)   # Compute epoch loss
        epoch_accuracy = correct / total                        # Compute epoch accuracy
        train_history['loss'].append(epoch_loss)                # Record training loss
        train_history['accuracy'].append(epoch_accuracy)        # Record training accuracy
        
        # Set model to Eval mode, Initialize variables
        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        # Validation loop
        with torch.no_grad():
            for X_val, y_val in test_loader:
                outputs = model(X_val)                                  # Forward pass
                val_loss = criterion(outputs, y_val)                    # Compute validation loss
                val_running_loss += val_loss.item() * X_val.size(0)     # Accumulate validation running loss
                
                _, val_predicted = torch.max(outputs, 1)                # Get validation predictions
                val_correct += (val_predicted == y_val).sum().item()    # Count correct validation predictions
                val_total += y_val.size(0)                              # Count total validation samples
        
        val_epoch_loss = val_running_loss / len(test_loader.dataset)    # Compute validation epoch loss
        val_epoch_accuracy = val_correct / val_total                    # Compute validation epoch accuracy
        train_history['validation_loss'].append(val_epoch_loss)         # Record validation loss
        train_history['validation_accuracy'].append(val_epoch_accuracy) # Record validation accuracy
        
        # Print statistics per epoch 
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_accuracy:.4f}')

        # Check for early stopping
        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            current_patience = 0  # Reset patience counter
            best_model = model  # Update best model
            best_train_history = train_history  # Update best training history
        else:
            current_patience += 1  # Increment patience counter

            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch+1} due to lack of improvement.')
                return best_model, best_train_history  # Return the best model and its training history
    return best_model, best_train_history  # Return the best model and its training history

# Evaluate Model
def evaluate_model(model, test_loader):
    """
    Evaluate the given model.
    
    Parameters:
        model (nn.Module): The neural network model to evaluate.
        test_loader (DataLoader): DataLoader for the test/validation data.
    
    Returns:
        None
    """

    # Set model to Eval mode, Initialize variables
    model.eval()
    y_pred_list = []

    with torch.no_grad():
        for X_batch, _ in test_loader:
            outputs = model(X_batch)            # Forward pass
            _, y_pred = torch.max(outputs, 1)   # Get predictions
            y_pred_list.append(y_pred.numpy())  # Store predictions

    y_pred = np.concatenate(y_pred_list)                                                # Concatenate all predictions
    y_test_numpy = y_test.to_numpy() if not isinstance(y_test, np.ndarray) else y_test  # Ensure y_test is a numpy array
    y_pred_flat = y_pred.flatten()                                                      # Flatten predictions

    accuracy = accuracy_score(y_test_numpy, y_pred_flat)        # Compute accuracy
    report = classification_report(y_test_numpy, y_pred_flat)   # Generate classification report

    # Print evaluation results
    print("Accuracy:", accuracy)
    print("Classification Report:\n", report)

## Train and evaluate base model

In [15]:
# Initialize the model, loss function, and optimizer
input_dim = X_train.shape[1]                # Input dimensions for the model, number of features in data
hidden_dim = 128                            # Number of neurons in the hidden layer    
output_dim = len(set(y_train.tolist()))     # Output dimensions for the model

# Instantiate the model
model = DroughtNet(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()                       # Define the loss function to be CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)    # Define the optimizer to be Adam, Learning rate to 0.001

# Train the base model and evaluate it
best_model, train_history = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=3)   # Train the model
evaluate_model(best_model, test_loader)                                                                  # Evaluate the model

# Save the trained base model
torch.save({
    'model_state_dict': model.state_dict(),  
    'optimizer_state_dict': optimizer.state_dict(),  
    'train_history': train_history,  
}, 'saved_model.pth') 


## Train and evaluate more complex model

In [12]:
# Define hidden dimensions and initialize the complex model
hidden_dims = [128, 64, 32]
model_complex = DroughtNetComplex(input_dim, hidden_dims, output_dim)   # Initialize the complex model
optimizer_complex = optim.Adam(model_complex.parameters(), lr=0.001)    # Define same optimizer for complex model

# Train/Evaluate the complex model
best_model_complex, train_history_complex = train_model(model_complex, train_loader, test_loader, criterion, optimizer_complex, num_epochs=3)
evaluate_model(model_complex, test_loader)

# Save the trained complex model
torch.save({
    'model_state_dict': model_complex.state_dict(),  
    'optimizer_state_dict': optimizer_complex.state_dict(), 
    'train_history': train_history_complex, 
}, 'saved_model2.pth')  


KeyboardInterrupt: 

## Random Search for Best Hidden Layer Size/Configuration

In [17]:
def generate_hidden_layer_choices(num_choices, min_layers, max_layers, min_neurons, max_neurons):
    """
    Generate a list of hidden layer configurations with random number of layers and neurons.

    Args:
        - num_choices (int): Number of different hidden layer configurations to generate.
        - min_layers (int): Minimum number of hidden layers.
        - max_layers (int): Maximum number of hidden layers.
        - min_neurons (int): Minimum number of neurons in each hidden layer.
        - max_neurons (int): Maximum number of neurons in each hidden layer.

    Returns:
        - hidden_layer_choices (list): List of hidden layer configurations.
    """
    hidden_layer_choices = []  # Initialize an empty list to store the hidden layer configurations
    
    # For number of hidden layer configurations
    for _ in range(num_choices):
        num_layers = random.randint(min_layers, max_layers)                             # Randomly choose the number of layers
        layers = [random.randint(min_neurons, max_neurons) for _ in range(num_layers)]  # Randomly choose neurons for each layer
        hidden_layer_choices.append(layers)                                             # Append the configuration to the list
    
    return hidden_layer_choices

In [24]:
num_choices = 100   # Number of different hidden layer configurations to generate
min_layers = 1      # Minimum number of hidden layers
max_layers = 5      # Maximum number of hidden layers
min_neurons = 8     # Minimum number of neurons in each hidden layer
max_neurons = 1024  # Maximum number of neurons in each hidden layer

# Generate hidden layer choices
hidden_layer_choices = generate_hidden_layer_choices(num_choices, min_layers, max_layers, min_neurons, max_neurons)

# Print the generated hyperparameter search space
print(f'Hyperparameter random search space: {hidden_layer_choices}')


Hyperparameter random search space: [[858, 939], [818, 195, 47, 788], [614, 142, 638], [179, 145, 356], [111, 201], [533, 614], [855, 468], [490], [632, 483, 143], [933, 667], [458, 414, 515], [731], [771, 638], [870, 853, 1005], [453, 490], [410, 504, 523, 536], [208, 31, 221, 277, 441], [768, 22, 808], [299, 95, 462, 805], [275, 428], [280, 115, 937, 670, 691], [537], [37, 720, 836, 387], [691, 154, 394, 997], [280, 15], [917, 62, 412], [780], [11, 700, 821, 268, 353], [252, 705, 20, 499], [650], [728], [36, 717, 592, 116, 872], [619, 905, 148, 791, 649], [642], [30, 1021, 185], [438, 516, 1016, 338, 131], [288, 150, 9, 283], [798, 967, 393, 924, 17], [882, 350, 971, 221, 386], [342, 712, 291, 946], [295, 392, 229, 791], [886, 705, 669], [861, 562, 178, 138], [509, 493, 309], [836, 623], [583, 222, 806, 756], [470, 329], [556, 11], [565, 492, 897], [163], [923, 551, 1016, 348, 179], [55], [367], [1007, 702], [363, 191, 753, 834], [880, 696, 715, 842], [500, 747, 255, 634], [428, 467,

In [13]:
# Initialize variables for search
num_searches = 10
best_model = None
best_val_accuracy = 0
best_hidden_dims = None

# Random search for num_searches
for i in range(num_searches):
    print(f"Random Search {i+1}/{num_searches}")                    # Print the current random search iteration
    hidden_dims = random.choice(hidden_layer_choices)               # Choose random hidden layer dimensions
    model = DroughtNetComplex(input_dim, hidden_dims, output_dim)   # Initialize the complex model
    optimizer = optim.Adam(model.parameters(), lr=0.001)            # Initialize the optimizer
    
    # Train the model with early stopping and get the best model and its training history
    best_model, train_history = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=3)
    val_accuracy = train_history['validation_accuracy'][-1]
    
    # Update the best model and accuracy if current validation accuracy is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model
        best_hidden_dims = hidden_dims

# Print the best validation accuracy and corresponding hidden dimensions
print(f"Best validation accuracy: {best_val_accuracy}")
print(f"Best hidden dimensions: {best_hidden_dims}")

Random Search 1/10


KeyboardInterrupt: 

In [ ]:
# Initialize the best model with the best hidden dimensions
model = DroughtNetComplex(input_dim, best_hidden_dims, output_dim)  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# Train/Evaluate the best model with more epochs and early stopping
best_model, train_history = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=100, patience=3) 
evaluate_model(model, test_loader)                                                                  

# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_history': train_history,
}, 'saved_best_model.pth')